In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
#drive.mount('/content/drive')
df=pd.read_excel("/content/drive/MyDrive/Module_Cloud _Data_ Engineering /TP3/diabetes.xlsx",sheet_name="data")

# Y : variable cible (diabète)
# Transformation en matrice numpy (format verticale) - seul reconnu par scikit-learn

In [ ]:
# X matrice des varibale explicatives les 8 première variables
#X=
# Spécifier la colonne de la variable à prédire (y) et les colonnes des variables explicatives (X)
y = df['class']  # Remplacez 'Nom_de_la_colonne_de_la_variable_a_predire' par le nom correct de votre colonne cible

# Sélectionnez les 8 premières colonnes comme variables explicatives (X)
X = df.iloc[:, 0:8]

# Afficher les premières lignes de X et y pour vérification
print(X.head())
print(y.head())

   preg  plas  pres  skin  insu  mass   pedi  age
0     6   148    72    35     0  33.6  0.627   50
1     1    85    66    29     0  26.6  0.351   31
2     8   183    64     0     0  23.3  0.672   32
3     1    89    66    23    94  28.1  0.167   21
4     0   137    40    35   168  43.1  2.288   33
0    tested_positive
1    tested_negative
2    tested_positive
3    tested_negative
4    tested_positive
Name: class, dtype: object


# Utilisation du module moel_slection de scikit-learn (sklearn) pour subdiviser l'échantillon en un échantillon  d'apprentissage et de validation

In [ ]:
from sklearn.model_selection  import train_test_split

# Subdivision des données – éch.test = 300 ; éch.app = 768 – éch.test = 468

In [ ]:
X_app,X_test,y_app,y_test= train_test_split(X,y,test_size = 300,random_state=0)
print(X_app.shape,X_test.shape,y_app.shape,y_test.shape)

(468, 8) (300, 8) (468,) (300,)


# Importer la classe LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression


# Création d'une instance de la classe

In [ ]:
lr = LogisticRegression(solver='liblinear')

# Algorithme de sélection de var.

In [ ]:
from sklearn.feature_selection import RFE
selecteur = RFE(estimator=lr)

# Lancer la recherche

In [ ]:
sol=selecteur.fit(X, y)

# Nombre de varariables sélectionnées

In [ ]:
print(sol.n_features_) # 4 = 8 / 2 variables sélectionnées

4


# Liste des variables sélectionnées

In [ ]:
print(sol.support_) # [ True  True False False False  True  True False]

[ True  True False False False  True  True False]


# Ordre de suppression

In [ ]:
print(sol.ranking_) # [1 1 2 4 5 1 1 3]

[1 1 2 4 5 1 1 3]


# Réduction de la base d'app. aux var. sélectionnées en utilisant le filtre booléen sol.support_

In [ ]:
X_new_app = X_app.loc[:, sol.support_]

#X_new_app = X_app[:,sol.support_]
print(X_new_app.shape) # (468, 4) 4 variables restantes

(468, 4)



# Construction du modèle sur les explicatives sélectionnées

In [ ]:
modele_sel = lr.fit(X_new_app,y_app)

# Réduction de la base test aux mêmes variables

In [ ]:
#X_new_test = X_test[:,sol.support_]
X_new_test = X_test.loc[:, sol.support_]


# Prédiction du modèle réduit sur l’éch. test

In [ ]:
y_pred_sel = modele_sel.predict(X_new_test)

# Evaluation

In [ ]:
from sklearn import metrics
print(metrics.accuracy_score(y_test,y_pred_sel)) # 0.787 # Aussi bien
#(presque, 0.793) que le modèle initial, mais avec moitié moins de variables.

0.7866666666666666


#Résulats sans supprision du data


In [ ]:
modele = lr.fit(X_app,y_app)
#X_new_test = X_test[:,sol.support_]
#X_new_test = X_test.loc[:, sol.support_]
y_pred = modele.predict(X_test)
print(metrics.accuracy_score(y_test,y_pred))

0.7933333333333333
